# The Thermal Expansion of MgO - Molecular Dynamics

Finally, we are going to calculate the volume of the MgO crystal, at different temperatures, by using molecular dynamics. 
First, run the calculation for T=100K, a window will open to show you the cell used for this calculation. Then animate the motion of the atoms by using the cell below.<br>
You will notice that as the atoms move, some of them seem to disappear. Analise the trajectory file frame by frame (by using the back/forward buttons) and understand where the atoms are going.<br>
Now uncomment (remove the '#' from) the third to the bottom line: <br><br>
MgO_SC_MD_out = cut(MgO_SC_MD, origo=(MgO_SC_MD.get_center_of_mass(scaled=True)),extend=1.01)
<br><br>
and run the MD calculation again. <font color=blue>What happens if you animate the motion of the atoms this time?</font>. Use this settings for the following calculations. Increase the T from 100K to 1000K in 100K intervals. As in the previous exercise you can do so by manually changing the T value in the input below, or you can write a for loop that will do that for you.


<br><br>

In [ ]:
#the imports first, always
import numpy as np
import re
import os
from ase.build import make_supercell
from ase.io import Trajectory
from ase.build import cut
from os import remove
from ase.spacegroup import crystal
from ase.visualize import view
from ase.calculators.gulp import GULP

#SELECT THE TEMPERATURE
T = 300
# generate the crystal starting from a geometry that is close to the optimised one
a = 4.1
#generate the conventional cell
MgO = crystal('MgO',basis=[[0.0,0.0,0.0],[0.5,0.5,0.5]],spacegroup=225,cellpar=[a, a, a, 90, 90, 90],primitive_cell=False)

#select the size of the supercell
supercell = 2
#create the supercell
MgO_SC_MD = make_supercell(MgO, supercell*np.identity(3))
#visualise the cell
view(MgO_SC_MD)
#name of the calculation
label='mgo_SC_MD_{}'.format(T)
#select the options for the calculations
options = [
'integrator leapfrog verlet',
'ensemble npt 0.005 0.005',
'temperature {}'.format(T),
'equil 0.50 ps',
'produ 0.50 ps',
'timestep 0.001 ps',
'sample 0.005 ps',
'iterations 5 ',
'dump every 5 noover {}.grs'.format(label)]
#run the calculation
calc = GULP(label=label,keywords='conp md', library='ionic.lib', options=options)
MgO_SC_MD.calc = calc
MgO_SC_MD.get_potential_energy()
#look for the optimised volume in the output
for line in reversed(list(open(label+'.got'))):
    a = re.match(r'\s*Cell volume\s*\S+\s*\S+\s*=\s*(\S+)\s*(\S+)', line)
    if a:
        V_final = a.group(2)
        print(' ({} K)  E ='.format(T),MgO_SC_MD.get_potential_energy(),'eV/(Unit cell) - Optimised Volume = ', a.group(2))
        break
#write the trajectory file to show the movement of the atoms at the temperature of the simulation
traj = Trajectory('{}K.trj'.format(label), 'w')    
for t in range(1,52):
    with open('{}_{}.grs'.format(label,t)) as o:
        pos_out = o.readlines()
        new_cell = np.zeros((3, 3), dtype=float)
        new_pos = np.zeros((MgO_SC_MD.get_number_of_atoms(), 3), dtype=float)
        for i,line in enumerate(pos_out):
            m =  re.match(r'vectors', line)
            if m:
                for j in range(0,3):
                    new_cell[j] = [float(x) for x in pos_out[i+j+1].split()]
            n = re.match(r'cartesian',line)
            if n:
                for j in range(0,MgO_SC_MD.get_number_of_atoms()):
                    first = pos_out[i+j+1].split()
                    first = first[2:5]
                    new_pos[j] = [float(x) for x in pos_out[i+j+1].split()[2:5]]    
        MgO_SC_MD.set_cell(new_cell)
        MgO_SC_MD.set_positions(new_pos)
        MgO_SC_MD_out = MgO_SC_MD
        MgO_SC_MD_out = cut(MgO_SC_MD, origo=(MgO_SC_MD.get_center_of_mass(scaled=True)),extend=1.01)
        traj.write(MgO_SC_MD_out)
traj.close()
#removing the files we don't need anymore to leave the directory clean
for i in range(1,52):
    if os.path.exists('./mgo_SC_MD_{}_{}.grs'.format(T,i)):
        os.remove('./mgo_SC_MD_{}_{}.grs'.format(T,i))
    else:
        break


In [ ]:
#use this cell to animate the motion of the atoms. You will have to change the name of the .trj file
! ase gui mgo_SC_MD_300K.trj 2> tmp

**Questions**:<br>
1. How is the cell we are using here different from the previous ones?<br>
    - Why do we need a larger cell?<br>
    - Is the cell we are using large enough? Explain why (think about the link between using a grid in reciprocal space and a supercell in real space).<br>
    - From your experience of computing an accurate Free Energy how large a cell should be used to perform reliably MD for MgO?<br>
    <br>
2. Replot your data from the quasi-harmonic approximation calculations as, cell volume per formula unit vs temperature, and add some points from the MD runs at a few suitable temperatures.<br>
    - How does the thermal expansion predicted by MD compare to that predicted by the quasi-harmonic approximation?<br>
    - Why do the two methods produce different answers? How does the difference depend on temperature?<br>
    
    - What would happen to the cell volume at very high temperature in the quasi-harmonic approximation and in the MD?<br>